# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
# basic
import pandas as pd
import numpy as np
import os
import re
import pycountry

# visualization
from matplotlib import pyplot as plt, font_manager
import seaborn as sns
plt.rcParams['font.family'] = 'NanumGothic'

import plotly
import plotly.express as px
import plotly.graph_objects as go

## 2. Load data

In [4]:
# load
train2 = pd.read_csv('../data/new_imputation_train2.csv')
test2 = pd.read_csv('../data/new_imputation_test2.csv')

train = pd.read_csv('../data/new_imputation_train.csv')
test = pd.read_csv('../data/new_imputation_test.csv')

## 3. Data Info

In [5]:
train2.isna().sum()

bant_submit                    0
business_unit                  2
customer_idx                   0
customer_type              42943
enterprise                     0
historical_existing_cnt    44115
idit_strategic_ver         53624
customer_job               18790
lead_desc_length               0
inquiry_type                 914
product_category               0
customer_position              0
response_corporate             0
expected_timeline              0
ver_cus                        0
ver_pro                        0
business_area              39933
business_subarea           52134
lead_owner                     0
is_converted                   0
customer_country3              0
continent                      0
dtype: int64

In [6]:
# 혹시나 짜른 데이터 중에 없는 데이터 찾기 또는 다른 값 반영되었나 확인
for i in train.columns:
    print(i)
    print(train2[i][~train2[i].isin(train[i].unique())])

bant_submit
Series([], Name: bant_submit, dtype: float64)
business_unit
50981    NaN
50982    NaN
Name: business_unit, dtype: object
customer_idx
555    13485
Name: customer_idx, dtype: int64
customer_type
0        End-Customer
1        End-Customer
2        End-Customer
3        End-Customer
5        End-Customer
             ...     
57621    End-Customer
57622    End-Customer
57623    End-Customer
57630    End-Customer
57631    End-Customer
Name: customer_type, Length: 10397, dtype: object
enterprise
Series([], Name: enterprise, dtype: int64)
historical_existing_cnt
Series([], Name: historical_existing_cnt, dtype: float64)
customer_job
Series([], Name: customer_job, dtype: object)
lead_desc_length
Series([], Name: lead_desc_length, dtype: int64)
inquiry_type
Series([], Name: inquiry_type, dtype: object)
product_category
Series([], Name: product_category, dtype: object)
customer_position
10254                             math and physics teacher
10255                                 

## 4. Data Info

In [7]:
train2.columns

Index(['bant_submit', 'business_unit', 'customer_idx', 'customer_type',
       'enterprise', 'historical_existing_cnt', 'idit_strategic_ver',
       'customer_job', 'lead_desc_length', 'inquiry_type', 'product_category',
       'customer_position', 'response_corporate', 'expected_timeline',
       'ver_cus', 'ver_pro', 'business_area', 'business_subarea', 'lead_owner',
       'is_converted', 'customer_country3', 'continent'],
      dtype='object')

In [8]:
train2['continent'].unique()

array(['Asia', 'Africa', 'South America', 'North America', 'Oceania',
       'Europe'], dtype=object)

In [9]:
test2['continent'].unique()

array(['South America', 'North America', 'Asia', 'Europe', 'Africa',
       'Oceania'], dtype=object)

In [10]:
# other은 그냥 결측치로 만들어 버리기
train2['inquiry_type'][train2['inquiry_type'] == 'other'] = np.nan
test2['inquiry_type'][test2['inquiry_type'] == 'other'] = np.nan

train2['inquiry_type'][train2['inquiry_type'] == 'other'] = np.nan
test2['inquiry_type'][test2['inquiry_type'] == 'other'] = np.nan

train2['inquiry_type'][train2['inquiry_type'].isin(['others', 'error'])] = np.nan
test2['inquiry_type'][test2['inquiry_type'].isin(['others', 'error'])] = np.nan

train2['product_category'][train2['product_category'].isin(['O/W', 'NOT-AVAILABLE'])] = np.nan
test2['product_category'][test2['product_category'].isin(['O/W', 'NOT-AVAILABLE'])] = np.nan

train2['customer_position'][train2['customer_position'].isin(['none', 'others', 'other'])] = np.nan
test2['customer_position'][test2['customer_position'].isin(['none', 'others', 'other'])] = np.nan

train2['expected_timeline'][train2['expected_timeline'] == 'Unknown'] = np.nan
test2['expected_timeline'][test2['expected_timeline'] == 'Unknown'] = np.nan

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/1367628559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train2['inquiry_type'][train2['inquiry_type'] == 'other'] = np.nan
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/1367628559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['inquiry_type'][test2['inquiry_type'] == 'other'] = np.nan
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/1367628559.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [11]:
train2.isna().sum()

bant_submit                    0
business_unit                  2
customer_idx                   0
customer_type              42943
enterprise                     0
historical_existing_cnt    44115
idit_strategic_ver         53624
customer_job               18790
lead_desc_length               0
inquiry_type                2324
product_category           20475
customer_position          26610
response_corporate             0
expected_timeline          30720
ver_cus                        0
ver_pro                        0
business_area              39933
business_subarea           52134
lead_owner                     0
is_converted                   0
customer_country3              0
continent                      0
dtype: int64

In [12]:
# partnership
partner1 = train.groupby(['customer_idx', 'lead_owner']).sum()['is_converted']
partner2 = train.groupby(['customer_idx', 'lead_owner']).count()['is_converted']
partner3 = train.groupby(['customer_idx', 'lead_owner']).sum()['is_converted']/train.groupby(['customer_idx', 'lead_owner']).count()['is_converted']

# merge & arrangae name
pt = pd.concat([partner1, partner2, partner3], axis = 1)
pt.columns = ['true_count', 'count', 'percent']
ppt = pt.reset_index()

# 명단 만들기
ppt2 = ppt[['customer_idx', 'lead_owner']][(ppt['count'] >= 2) & (ppt['percent'] >= 0.5)]
ppt2['partnership'] = 1

# train, test에 합치기
train3 = pd.merge(train2, ppt2, how = 'left', on = ['customer_idx', 'lead_owner'])
test3 = pd.merge(test2, ppt2, how = 'left', on = ['customer_idx', 'lead_owner'])

# 없는 값 무조건 0으로 채워야됨
train3['partnership'][train3['partnership'].isna()] = 0
test3['partnership'][test3['partnership'].isna()] = 0

train2 = train3.copy()
test2 = test3.copy()

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/2056487621.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train3['partnership'][train3['partnership'].isna()] = 0
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/2056487621.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test3['partnership'][test3['partnership'].isna()] = 0


In [13]:
# 판매자와 product 융합
dsa = train.groupby(['product_category', 'lead_owner']).sum()['is_converted']
dsa2 = train.groupby(['product_category', 'lead_owner']).count()['is_converted']
dsa3 = pd.merge(dsa, dsa2, how = 'outer', on = ['product_category', 'lead_owner']).reset_index()
dsa3['fan_mae_king'] = dsa3['is_converted_x'] / dsa3['is_converted_y']

# 상위 15개만 선정하기
dsa4 = dsa3.sort_values(['product_category', 'fan_mae_king'], ascending = False).groupby('product_category').head(15)

# 명단 만들기
dsa4['fan_mae_king_yn'] = 1
dsa5 = dsa4[['product_category', 'lead_owner', 'fan_mae_king_yn']]
# train, test에 합치기
train3 = pd.merge(train2, dsa5, how = 'left', on = ['product_category', 'lead_owner'])
test3 = pd.merge(test2, dsa5, how = 'left', on = ['product_category', 'lead_owner'])

# 없는 값 무조건 0으로 채워야됨
train3['fan_mae_king_yn'][train3['fan_mae_king_yn'].isna()] = 0
test3['fan_mae_king_yn'][test3['fan_mae_king_yn'].isna()] = 0

train2 = train3.copy()
test2 = test3.copy()

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/2752353905.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train3['fan_mae_king_yn'][train3['fan_mae_king_yn'].isna()] = 0
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/2752353905.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test3['fan_mae_king_yn'][test3['fan_mae_king_yn'].isna()] = 0


In [14]:
# country와 product 융합
da = train.groupby(['customer_country3', 'product_category']).sum()['is_converted']
da2 = train.groupby(['customer_country3', 'product_category']).count()['is_converted']
da3 = pd.merge(da, da2, how = 'outer', on = ['customer_country3', 'product_category']).reset_index()
da3['percentage'] = da3['is_converted_x'] / da3['is_converted_y']

# 그룹 나누기 (cnt = 14 : 75%)
### 0: otherwise 
### 1: cnt >= 14, 확률 > 0
### 2: 확률 > 0.5
### 3: cnt >= 14, 확률 > 0.5
da3['country_to_product'] = 0
da3['country_to_product'][(da3['is_converted_y'] >= 14) & (da3['percentage'] > 0)] = 1
da3['country_to_product'][(da3['percentage'] > 0.5)] = 2
da3['country_to_product'][(da3['is_converted_y'] >= 14) & (da3['percentage'] > 0.5)] = 3

# 기준 변수 생성하기
ctpd = da3[['customer_country3', 'product_category', 'country_to_product']]

# train, test에 합치기
train3 = pd.merge(train2, ctpd, how = 'left', on = ['customer_country3', 'product_category'])
test3 = pd.merge(test2, ctpd, how = 'left', on = ['customer_country3', 'product_category'])

# 없는 값 무조건 0으로 채워야됨
train3['country_to_product'][train3['country_to_product'].isna()] = 0
test3['country_to_product'][test3['country_to_product'].isna()] = 0

train2 = train3.copy()
test2 = test3.copy()

/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/639592810.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da3['country_to_product'][(da3['is_converted_y'] >= 14) & (da3['percentage'] > 0)] = 1
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/639592810.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da3['country_to_product'][(da3['percentage'] > 0.5)] = 2
/var/folders/kv/46dvhwp932v4b692v761hjdc0000gn/T/ipykernel_15356/639592810.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [15]:
train2.isna().sum()

bant_submit                    0
business_unit                  2
customer_idx                   0
customer_type              42943
enterprise                     0
historical_existing_cnt    44115
idit_strategic_ver         53624
customer_job               18790
lead_desc_length               0
inquiry_type                2324
product_category           20475
customer_position          26610
response_corporate             0
expected_timeline          30720
ver_cus                        0
ver_pro                        0
business_area              39933
business_subarea           52134
lead_owner                     0
is_converted                   0
customer_country3              0
continent                      0
partnership                    0
fan_mae_king_yn                0
country_to_product             0
dtype: int64

In [16]:
# 데이터 저장
train2.to_csv('train_final.csv', index = False)
test2.to_csv('test_final.csv', index = False)